In [1]:
## Interface testing

import tkinter as tk
import tkinter.font as tkFont
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from yahoo_fin.stock_info import get_data
from datetime import datetime
from keras.models import Sequential, load_model

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

class App:
    def __init__(self, master):

        frame = tk.Frame(master)

        #Setting dataset
        ticker_list = ["amzn", "eth-usd", "ba", "^gspc", "nflx", "fb", "tsla", "ko", "nsrgy", "pep", "mnst", "wmt", "t", "vz", "gm", "btc-usd", "mcd", "qsr", "msft", "sne"]
        historical_datas = {}
        for ticker in ticker_list:
            historical_datas[ticker] = get_data(ticker, index_as_date=True, interval="1d")

        self.df = historical_datas

        self.title_style = tkFont.Font(family="Arial", size=24)
        self.label_title = tk.Label(frame, text="Stocks Prediction System", font=self.title_style)
        self.label_title.pack()

        self.frame_search = tk.Frame(frame, height=40)
        self.frame_space = tk.Frame(frame)
        self.frame_fig = tk.Frame(frame)

        self.stock_selected = tk.StringVar(self.frame_search)
        self.stock_selected.set("tsla") # default value

        #Define Figure
        fig = Figure(figsize=(8, 3))
        self.ax = fig.add_subplot(111)
        self.ax.locator_params(axis="x", nbins=5)
        #self.line, = self.ax.plot(self.df[self.stock_selected.get()]['close'])
        self.canvas = FigureCanvasTkAgg(fig,master=self.frame_fig)
        self.canvas.show()
        self.canvas.get_tk_widget().pack(anchor=tk.NW)


        self.w = tk.OptionMenu(self.frame_search, self.stock_selected, "amzn", "eth-usd", "ba", "^gspc", "nflx", "fb",
                               "tsla", "ko", "nsrgy", "pep", "mnst", "wmt", "t", "vz", "gm", "btc-usd", "mcd",
                               "qsr", "msft", "sne", command=self.make_pred)
        self.w.pack(side=tk.RIGHT, fill=tk.BOTH, expand=tk.TRUE)
        self.label_m = tk.Label(self.frame_search, text="Stock", width=10)
        self.label_m.pack(side=tk.RIGHT)
        self.label_space = tk.Label(self.frame_search, text="", width=5)
        self.label_space.pack(side=tk.RIGHT)

        self.time_variable = tk.StringVar(self.frame_search)
        self.time_variable.set("Forever") # default value
        self.w = tk.OptionMenu(self.frame_search, self.time_variable, "Forever", "2Y", "1Y", "6m", command=self.make_pred)
        self.w.pack(side=tk.RIGHT, fill=tk.BOTH, expand=tk.TRUE)
        self.label_m = tk.Label(self.frame_search, text="Range", width=10)
        self.label_m.pack(side=tk.RIGHT)

        self.make_pred(self)

        ## Pack

        self.frame_search.pack(fill=tk.X, padx=5, pady=5);
        self.frame_space.pack(fill=tk.X, padx=5, pady=5);
        self.frame_fig.pack(side=tk.LEFT, padx=5, pady=5);

        frame.pack()

    @staticmethod
    def datetime_to_timestamp(x):
            return datetime.strptime(x.strftime('%Y%m%d'), '%Y%m%d')

    def make_pred(self, var):

        data = self.df[self.stock_selected.get()]
        data.index = data.index.to_series().apply(self.datetime_to_timestamp)

        if self.time_variable.get() == "Forever":
            data = self.df[self.stock_selected.get()]
        elif self.time_variable.get() == "2Y":
            data = data.loc["2019-01-01":]
        elif self.time_variable.get() == "1Y":
            data = data.loc["2020-01-01":]
        elif self.time_variable.get() == "6m":
            data = data.loc["2020-06-01":]

        new_dataset = pd.DataFrame(index=range(0,len(data)),columns=['date','adjclose'])

        for i in range(0,len(data)):
            new_dataset['date'][i] = data.index[i]
            new_dataset['adjclose'][i]=data["adjclose"][i]

        final_set = new_dataset.values

        train_test_split = int(len(final_set)*.8)
        train_data = final_set[:train_test_split]
        valid_data = final_set[train_test_split:]

        new_dataset.index = new_dataset.date
        new_dataset.drop('date', axis=1, inplace=True)

        final_set = new_dataset.values

        scaler = MinMaxScaler(feature_range=(0,1))
        scaled_data = scaler.fit_transform(final_set)

        inputs_data=new_dataset[len(new_dataset)-len(valid_data)-60:].values
        inputs_data=inputs_data.reshape(-1,1)
        inputs_data=scaler.transform(inputs_data)

        model= load_model("new_model.h5")

        X_data=[]
        for i in range(60,inputs_data.shape[0]):
            X_data.append(inputs_data[i-60:i,0])
        X_data = np.array(X_data)
        X_data = np.reshape(X_data,(X_data.shape[0],X_data.shape[1],1))
        predicted_closing_price = model.predict(X_data)

        test_performance = predicted_closing_price
        predicted_closing_price = scaler.inverse_transform(predicted_closing_price)

        train_data=new_dataset[:train_test_split]
        valid_data=new_dataset[train_test_split:]
        valid_data['Predictions']=predicted_closing_price
        plt.figure(figsize=(16,8))
        plt.plot(train_data["adjclose"], label='Close Data')
        plt.plot(valid_data[['adjclose',"Predictions"]], label='Close Data Prediction')
        plt.legend()
        plt.show()
        self.ax.clear()
        self.ax.plot(train_data["adjclose"], label='Close Data')
        self.ax.plot(valid_data[['adjclose',"Predictions"]], label='Close Data Prediction')
        self.canvas.draw()


root = tk.Tk()
app = App(root)
root.mainloop()


#####IMPLEMENT PERFORMANCE EVALUATION

c:\users\joao-\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:46: MatplotlibDeprecationWarning: The FigureCanvasTk.show function was deprecated in version 2.2. Use FigureCanvasTk.draw instead.
C:\Users\joao-\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
c:\users\joao-\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


<Figure size 1600x800 with 1 Axes>

<Figure size 1600x800 with 1 Axes>